# Data cleaning process
Let's define a market which:
* Takes Bloomberg input with multiple stocks
* Sets up a dictionanry of dataframes including each stock
* Handles missing values:
    * Remove stocks that start after 20040101
    * Remove holidays

In [ ]:
from bokeh.charts import Bar, output_file, output_notebook, show, reset_output
from collections import OrderedDict
from datetime import datetime
from pandas import Series, DataFrame
from pandas.tseries.offsets import BDay

import csv
import numpy as np
import pandas as pd
import pandas.io.data
import time

## Market definition

In [ ]:
class Market:
    def __init__(self, file='../donnees/raw/market.csv'):
        self.marketFile=file
        self.stockNames = self.getStockNames()
        self.stocks = self.getStocks()
        
        self.features = {}
        
        self.logReturns = None
    
    # LOADING PROCESS
    # Read file including equity values:
    # nb1 Equity;;;;;;;nb2 Equity;;;;;;;...
    # Date;PX_LAST;PX_OPEN;PX_HIGH;PX_LOW;PX_VOLUME;;Date...
    # Retrieve names
    def getStockNames(self):
        stockNames = []
        with open(self.marketFile) as csvfile:
            spamreader = csv.reader(csvfile, delimiter=';')
            firstRow = True
            for row in spamreader:
                if firstRow:
                    for i, stockName in enumerate(row):
                        if i % 7 == 0:
                            toBeAppended = stockName.replace(' Equity','').replace(' ', '')
                            stockNames.append(toBeAppended)
                    firstRow = not firstRow
        print('Stock names loaded')
        return(stockNames)
    
    # Divide in several data frames
    # 1 data frame per stock
    def getStocks(self):
        stocks = {}
        i = 0
        for index, elt in enumerate(self.stockNames):
            tmp = pd.read_csv(filepath_or_buffer=self.marketFile,
                              sep=';',
                              header=1,
                              names=['Date', 'PX_LAST', 'PX_OPEN', 'PX_HIGH', 'PX_LOW', 'PX_VOLUME'],
                              usecols=[j for j in range(i, i + 6)],
                              low_memory=False).dropna(how='all')
            tmp['Date'] = pd.to_datetime(tmp['Date'], format="%d/%m/%Y")
            tmp = tmp.set_index(['Date']).ix[:-1]
            stocks[elt] = tmp
            i += 7
        print('Stocks loaded')
        return(stocks)
    
    def joinFeatures(self):
        # Concatenate stock frames by index. Pandas figures out which are the missing values, and just fill them next.
        features = ['PX_LAST', 'PX_OPEN', 'PX_HIGH', 'PX_LOW', 'PX_VOLUME']
        for feature in features:
            frames = []
            for stockName, stock in self.stocks.iteritems():
                frames.append(stock.rename(columns={feature: stockName})[stockName])
            self.features[feature] = pd.concat(frames, axis = 1).fillna(method='pad')
    
    ##### ADDITIONAL FEATURE #####
    def addLogReturns(self, feature):
        tmp = self.features[feature].pct_change()
        self.logReturns =  np.log(1 + tmp)[1:]
        return True

## Market loading

In [ ]:
cac40 = Market(file='../donnees/raw/donnees_CAC.csv')

In [ ]:
globalMarket = Market(file='../donnees/raw/market.csv')

Compare two markets

In [ ]:
set(cac40.stockNames).difference(set(globalMarket.stockNames).intersection(set(cac40.stockNames)))

{'LHN', 'NOKIA'}: French stocks that are not in European stocks

In [ ]:
len(globalMarket.stockNames)

## Start date formatting

### Analysis

184 stocks at disposal.

Let's find out start and end dates for each stock.

Why? Because further modelization requires that stocks we decided to involve have the same trading days.

In [ ]:
# Start dates to know which stock has to be discarded
## Data formatting to match Bokeh specs
reset_output()

startDates = {}
for stockName, stock in globalMarket.stocks.iteritems():
    startDate = pd.to_datetime(str(stock.index.values[0])).strftime('%Y-%m-%d')
    if startDate in startDates: startDates[startDate] += 1
    else: startDates[startDate] = 1
tmp = {}
tmp['dates'] = []
tmp['number'] = []
for startDate, number in startDates.iteritems():
    tmp['dates'].append(startDate)
    tmp['number'].append(number)
p = Bar(tmp, values='number', label='dates', background_fill_color="#E8DDCB", legend=None)
output_notebook()
show(p)

In [ ]:
# End dates to know which stock has to be discarded
endDates = {}
for stockName, stock in globalMarket.stocks.iteritems():
    endDate = pd.to_datetime(str(stock.index.values[len(stock.index.values)-1])).strftime('%Y-%m-%d')
    if endDate in endDates: endDates[endDate] += 1
    else: endDates[endDate] = 1
endDates

Current stocks are still trading today.

### Synthesis

From both start and end dates results, a rule of thumb is used: stocks starting after 20040101 are discarded, meaning we're getting rid of 18% of the stocks. Data completeness is favored compared to stock completeness => Relevant for next machine learning implementation

In [ ]:
startDate=datetime(2004, 1, 1)
stocksToDiscard=[]
for stockName, stock in globalMarket.stocks.iteritems():
    if pd.to_datetime(str(stock.index.values[0])) <= startDate: globalMarket.stocks[stockName] = stock[startDate:]
    else: stocksToDiscard.append(stockName)
for stockToDiscard in stocksToDiscard:
    del globalMarket.stocks[stockToDiscard]
    globalMarket.stockNames.remove(stockToDiscard)

In [ ]:
len(globalMarket.stockNames)

## Missing days handling

### Analysis

Despite stocks removal, irregularities remain because of trading day specifications in each country.
Let's track them.

In [ ]:
reset_output()

lengths = {}
for stockName, stock in globalMarket.stocks.iteritems():
    if stock.shape[0] not in lengths.keys(): lengths[stock.shape[0]] = 1
    else: lengths[stock.shape[0]] += 1

tmp = {}
tmp['dates'] = []
tmp['number'] = []
for startDate, number in lengths.iteritems():
    tmp['dates'].append(startDate)
    tmp['number'].append(number)
p = Bar(tmp, values='number', label='dates', background_fill_color="#E8DDCB", legend=None)

output_notebook()
show(p)

Reference (i.e who traded the most) seems to be 3201 trading days from 20040101 to 20160629.

### Synthesis

Hypothesis retained: discard a day if one of the stocks did not trade. It is a rough constraint, but it enables not to introduce a bias some days due to holidays in some countries while others are trading normally.

Let's find out which days are holidays for some countries, not others

In [ ]:
stocksPassed = []
daysToDiscard = []
for stockNameX, stockX in globalMarket.stocks.iteritems():
    stocksPassed.append(stockNameX)
    for stockNameY, stockY in globalMarket.stocks.iteritems():
        if stockNameY not in stocksPassed:
            symetricDifference = list(set(stockX.index.values).symmetric_difference(set(stockY.index.values)))
            for elt in symetricDifference: daysToDiscard.append(elt)
daysToDiscard = list(set(daysToDiscard))

In [ ]:
daysToDiscard

Holidays are clearly identified: e.g. May 1st for French stocks.

In [ ]:
len(daysToDiscard)

Remove bogus days

In [ ]:
for stockName, stock in globalMarket.stocks.iteritems():
    globalMarket.stocks[stockName] = stock.drop(pd.to_datetime(daysToDiscard), errors='ignore')

## Data frame formatting
Split stocks dict into data frames for each feature

In [ ]:
globalMarket.joinFeatures()

Append interest variable: log return of PX_LAST

In [ ]:
globalMarket.addLogReturns(feature='PX_LAST')

## Conclusion and saving
In the end, 35 stocks have been discarded. Remaining have been cleaned according to above rules of thumbs.

In [ ]:
features = ['PX_LAST', 'PX_OPEN', 'PX_HIGH', 'PX_LOW', 'PX_VOLUME']

In [ ]:
for feature in features: globalMarket.features[feature].to_csv(path_or_buf='../donnees/clean/' + feature + '.csv',
                                                               sep=';',
                                                               index_label='Date')

In [ ]:
globalMarket.logReturns.to_csv(path_or_buf='../donnees/clean/RET_PX_LAST.csv',
                               sep=';',
                               index_label='Date')